# RQ3 Notebook

In [1]:
!pip install python-igraph
!pip install implicit

     |████████████████████████████████| 3.2MB 21.6MB/s 
     |████████████████████████████████| 1.1MB 16.1MB/s 
  Created wheel for implicit: filename=implicit-0.4.4-cp36-cp36m-linux_x86_64.whl size=3419454 sha256=2e2d474bb88858bcc3535d5cec55913b8a47d42ebbc321133efcfe3e1891a3ef
  Stored in directory: /root/.cache/pip/wheels/bf/d4/ec/fd4f622fcbefb7521f149905295b2c26adecb23af38aa28217
Successfully built implicit


In [3]:
import hashlib
from collections import defaultdict 
import igraph
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix
from collections import Counter
import implicit
import math

from google.colab import drive
drive.mount('/content/drive/')
PATH = '/content/drive/My Drive/Colab Notebooks/IR-WA_FinalProject/data/'

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [4]:
df = pd.read_csv(f"{PATH}final_Tweets.csv")
df.head()

,created_at,entities.hashtags,favorite_count,full_text,id,retweet_count,user.id,user.name
0,2020-11-11,[],4,International friendly roundup: Finland stun F...,1326667371730378753,1,16042794,Guardian US
1,2020-11-11,[],11,When Joe Biden formally takes over the preside...,1326666012142526466,5,16042794,Guardian US
2,2020-11-11,[],4,New Yorker fires Jeffrey Toobin after he repor...,1326663505454510081,1,16042794,Guardian US
3,2020-11-11,[],8,One week on: how has Donald Trump handled losi...,1326661105498796032,1,16042794,Guardian US
4,2020-11-11,[],13,France pays tribute to six-year-old resistance...,1326659924278046728,6,16042794,Guardian US


## Adjecency Matrix

In [5]:
count = 0
community_members = []
adj_mat = np.zeros((1,1))
for i in range(len(df)):
    tweettext = df.loc[i].full_text
    if tweettext.startswith("RT @") == True:
        count+=1
        user1 =df.loc[i]['user.name']
        user2 = tweettext.split(' ')[1][1:]
        user1_hash = int(hashlib.sha1(user1.encode("utf-8")).hexdigest(), 16) % (10 ** 8)
        user2_hash = int(hashlib.sha1(user2.encode("utf-8")).hexdigest(), 16) % (10 ** 8)
        if user1_hash == user2_hash:
            print('both users are same')
        if user1_hash not in community_members:
            community_members.append(user1_hash)
            u1 = len(community_members)-1
            adj_mat[u1][u1] = 0
            adj_mat = np.append(adj_mat, [[0]]*adj_mat.shape[0],1)
            adj_mat = np.append(adj_mat, [[0]*adj_mat.shape[1]], 0)

        else:
            u1 = community_members.index(user1_hash)
        if user2_hash not in community_members:
            community_members.append(user2_hash)
            u2 = len(community_members)-1
            adj_mat[u2][u2] =0
            adj_mat = np.append(adj_mat, [[0]]*adj_mat.shape[0],1)
            adj_mat = np.append(adj_mat, [[0]*adj_mat.shape[1]], 0)
        else:
            u2 = community_members.index(user2_hash)
        adj_mat[u1][u2]=1
        adj_mat[u2][u1]=1
        
##adj_mat *= np.tri(*adj_mat.shape) ???

In [6]:
# Create the graph
G = igraph.Graph.Adjacency(adj_mat.tolist())

## Fins all nodes at distance 2

In [7]:
def find_nodes_at_distance_2(graph):
    """
    starting from a graph this function returns all the nodes at distance 2
    """
    

    all_potential_recommendations = set()
    
    for n1 in graph.vs:
        
        # all the nodes at distance 1
        nodes_at_most_distant_1 = set(graph.neighborhood(vertices=n1, order=1, mode='ALL'))

        # all the nodes at distance 1 and distance 2

        nodes_at_most_distant_2 = set(graph.neighborhood(vertices=n1, order=2, mode='ALL'))
        
        # only the nodes at distance 2
        only_nodes_at_distance_2 = nodes_at_most_distant_2 - nodes_at_most_distant_1
        
        
        # check if empty set
        if len(only_nodes_at_distance_2) > 0:           
            for n2 in only_nodes_at_distance_2:
                
                # since n1 is an igraph vertex object, we need to extract the id
                n1_index = n1.index
                
                all_potential_recommendations.add((n1_index, n2))
            
    return all_potential_recommendations

In [8]:
all_potential_recommendations = find_nodes_at_distance_2(G)

## Split the data set

In [9]:
#Test size
p = .2

# graphsize
N = len(G.es)

# idxs of all the edges
all_idxs = range(N)

# sample idxs of edges through the function "choice"
test_idxs = np.random.choice(a = all_idxs,size = int(N*p))

In [10]:
# Split dataset
ground_truth = set()
trainset = set()

for idx, one_edge in enumerate(G.es):
    
    # take n1 and n2 idx from one_edge, that is an igraph edge *object*
    n1 = one_edge.source
    n2 = one_edge.target

    if idx in test_idxs:        
        ground_truth.add((n1, n2, 1))
    else:     
        trainset.add((n1, n2, 1))

In [11]:
for rec in all_potential_recommendations:
    
    # add to ground truth also the potential nodes
    n1 = rec[0]
    n2 = rec[1]
    
    ground_truth.add((n1,n2,0))

In [70]:
ground_truth_d = defaultdict(list)

for n1,n2,w in ground_truth:
  ground_truth_d[n1].append(n2) 

## Create Prediction Graph

In [12]:
list_edges = []

for u,v,w in ground_truth:
  list_edges.append((u,v))

for u,v,w in trainset:
  list_edges.append((u,v))

In [13]:
G_pred = igraph.Graph()
G_pred.add_vertices(len(list_edges))
G_pred.add_edges(list_edges)

## Adamic-Adar

In [14]:
def compute_ADA(u,v,graph):
  
  # set neighbors u
  outlinks_u = graph.neighbors(u)

  # set neighbors v
  outlinks_v = graph.neighbors(v)

  # Set of common links
  bridges = set(outlinks_u).intersection(set(outlinks_v))

  # Degrees of set
  degree_list = [graph.degree(node) for node in bridges]

  # Reciprocal sum
  out = [1./np.log2(d+1) for d in degree_list]

  return out 

In [15]:
compute_ADA(1,2,G)

[0.227670248696953]

In [16]:
def predict_ADA(nodes_tuples, G):
  """
  Given a Graph and a list of node tuples computes the ADA measurement
  Return ada score per tuple, dict of dict
  """
  out = defaultdict(dict)
  for u,v in nodes_tuples:
    ADA_score = compute_ADA(u,v,G)
    out[u][v] = ADA_score

  return out

In [17]:
ada_predictions = predict_ADA(all_potential_recommendations,G)

In [61]:
for n in ada_predictions:
  pred_n = ada_predictions[n]
  pred_n = {k: v for k, v in sorted(pred_n.items(), key=lambda item: item[1], reverse = True)}
  ada_predictions[n] = pred_n

# ALS

In [18]:
# Get adjancency matrix
M =  G.get_adjacency().data
M = csr_matrix(M)

# Train the algorithm
model = implicit.als.AlternatingLeastSquares(factors = 10, calculate_training_loss= True, iterations = 10)

model.fit(M)

GPU training requires factor size to be a multiple of 32. Increasing factors from 10 to 32.


In [19]:
def predict_ALS(testset, model):
  """
  predict for a dict of dict of observations the score for adding a new link
  """

  all_predictions = defaultdict(dict)

  for n1, n2 in testset:
    
    # get factors
    factors_n1 = model.user_factors[n1,:]
    factors_n2 = model.item_factors[n2,:]

    # prediction score link
    score = np.dot(factors_n1,factors_n2)

    all_predictions[n1][n2] = score

  return all_predictions

In [20]:
als_predictions = predict_ALS(all_potential_recommendations, model)

In [58]:
# Sort
for n in als_predictions:
  pred_n = als_predictions[n]
  pred_n = {k: v for k, v in sorted(pred_n.items(), key=lambda item: item[1], reverse = True)}
  als_predictions[n] = pred_n

## PageRank

We have created a new graph named ```G_pred``` which contains the original graph plus new **edges** that connect nodes that are at a distance of two that previously were not connected. The PagerRank value for this connections corresponds to the predicted value.



In [21]:
# Node to node pairs with respective page rank
page_rank_dic = defaultdict(dict)

for u,v,w in ground_truth:
  page_rank_dic[u][v] = 0.0

for u,v,w in trainset:
  page_rank_dic[u][v] = 0.0

In [27]:
for n1 in page_rank_dic:

  # Compute page rank
  page_rank_n = G_pred.personalized_pagerank(reset_vertices = n1)

  # Get non-zero ranks nodes
  non_zero_rank_nodes =[i for i,n_ in enumerate(page_rank_n) if n_ != 0] 
  
  # Dict node-page rank score
  dict_zero_rank_nodes = {i:page_rank_n[i] for i in non_zero_rank_nodes}

  # Sort
  dict_zero_rank_nodes = {k: v for k, v in sorted(dict_zero_rank_nodes.items(), key=lambda item: item[1], reverse = True)}
  
  for n2 in dict_zero_rank_nodes.keys():  
    page_rank_dic[n1][n2] = page_rank_n[n2]

```page_rank_dic``` contains the page rank and page rank predictions for all the nodes in the Graph. We can select a pair from list ```all_potential_recommendations``` to se the predicted value. The first tuple in ```all_potential_recommendations``` are the nodes 1246 and 1065.

In [23]:
n1 = 1246
n2 = 1100

pr_pred = page_rank_dic[n1][n2]

print(f"The predicted Page Rank Score for nodes {n1} and {n2} is : {pr_pred}")

The predicted Page Rank Score for nodes 1246 and 1100 is : 0.002975678285873392


# NDCG

In [87]:
def ndcg(ground_truth_r, link_predictions, k):
  avg_ndcg = 0
  count = 0

  dcg = 0 
  for n1 in link_predictions:
    n_predict = len(link_predictions[n1])
    n_ground = len(ground_truth_r[n1])

    n = min(max(n_predict, n_ground), k)

    denom = np.arange(1,n+1)
    denom = np.log2(denom + 1)
    
    gains = 1/denom

    dcg_mask = [0]*n

    i = 0
    for n2 in link_predictions[n1]:
      if n2 in ground_truth_r[n1]:
        dcg_mask[i] = 1
      else:
        dcg_mask[i] = 0
      i += 1
      if i >= n:
        break
    
    dcg += sum([g*m for g,m in zip(gains, dcg_mask)])
    count +=1
   
  ndcg = dcg/count

  return ndcg

## Adamic-Adar

In [90]:
ndcg(ground_truth_d, ada_predictions, k = 10)

3.9267794686802455

## ALS NDCG

In [89]:
ndcg(ground_truth_d, als_predictions, k = 10)

5.676756267543735

## Page Rank  NDCG

In [ ]:
# Create needed arguments 
ground_truth_r = defaultdict(list)
link_predictions_r = defaultdict(list)

edge_list = G.get_edgelist()

for n1 in page_rank_dic:
  n2_list = page_rank_dic[n1].keys()

  for n2 in n2_list:
    if (n1, n2) in edge_list:
      ground_truth_r[n1].append(n2)
    elif (n1, n2) in all_potential_recommendations:
      link_predictions_r[n1].append(n2)

In [91]:
ndcg(ground_truth_d, link_predictions_r, k = 10)

0.0478